In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import DemoMouseEnhancers
from genomic_benchmarks.models.torch_cnn import CNN
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config 


## Config

In [12]:
config = {
    "use_padding": True,
    "run_on_gpu": True,
    "dataset": DemoMouseEnhancers,
    "number_of_classes": 2,
    "dataset_version": 0,
    "force_download": False,
    "epochs": 15,
    "embedding_dim": 100,
    "batch_size": 32,
#   vocabulary that is not present in the training set but is present in the test set
    "vocab_to_add": [],
}
check_config(config)

config is correct


## Choose the dataset

In [13]:
get_dataset_fn = config["dataset"]
train_dset = get_dataset_fn('train', force_download=config["force_download"], version=config["dataset_version"])

Unzipping...Done.


## Tokenizer and vocab

In [14]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 8
{'N': 5, '<pad>': 7, 'T': 3, 'C': 4, 'A': 2, '<eos>': 6, 'G': 1, '<bos>': 0}


## Dataloader and batch preparation

In [15]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  4707
not all sequences are of the same length


## Model

In [16]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

## Training

In [17]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0
Train metrics: 
 Accuracy: 65.4%, Avg loss: 0.634368 

Epoch 1
Train metrics: 
 Accuracy: 56.3%, Avg loss: 0.669674 

Epoch 2
Train metrics: 
 Accuracy: 67.5%, Avg loss: 0.626424 

Epoch 3
Train metrics: 
 Accuracy: 68.4%, Avg loss: 0.635009 

Epoch 4
Train metrics: 
 Accuracy: 74.1%, Avg loss: 0.608837 

Epoch 5
Train metrics: 
 Accuracy: 72.0%, Avg loss: 0.609211 

Epoch 6
Train metrics: 
 Accuracy: 77.8%, Avg loss: 0.596288 

Epoch 7
Train metrics: 
 Accuracy: 77.6%, Avg loss: 0.596530 

Epoch 8
Train metrics: 
 Accuracy: 77.9%, Avg loss: 0.589761 

Epoch 9
Train metrics: 
 Accuracy: 79.1%, Avg loss: 0.586003 

Epoch 10
Train metrics: 
 Accuracy: 79.5%, Avg loss: 0.591736 

Epoch 11
Train metrics: 
 Accuracy: 78.6%, Avg loss: 0.587679 

Epoch 12
Train metrics: 
 Accuracy: 77.4%, Avg loss: 0.590318 

Epoch 13
Train metrics: 
 Accuracy: 80.4%, Avg loss: 0.583303 

Epoch 14
Train metrics: 
 Accuracy: 80.0%, Avg loss: 0.581481 



## Testing

In [18]:
test_dset = get_dataset_fn('test', force_download=config["force_download"], version=config["dataset_version"])
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

model.test(test_loader)

Unzipping...Done.
test_loss  4.790966272354126
num_batches 8
correct 189
size 242
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.598871 

